In [66]:
from collections import namedtuple
import sympy
from sympy import Matrix, det, symbols, sin, cos, pi
from sympy.physics.units import speed_of_light, meter, second, convert_to, radian
import IPython.display

In [79]:
TrackingWheelCfg = namedtuple('TrackingWheelCfg', ['v', 'theta_rad', 'radius_meters'])

Twist = namedtuple("Twist", ["dx", "dy", "dtheta"])
Pose = namedtuple("Pose", ["x", "y", "theta"])

In [26]:
    acfg1 = TrackingWheelCfg(Matrix([['v_x1'], ['v_y1']]) * meter, symbols('o_1') * radian, symbols('r_1')* meter)
    acfg2 = TrackingWheelCfg(Matrix([['v_x2'], ['v_y2']]) * meter, symbols('o_2') * radian, symbols('r_2')* meter)
    acfg3 = TrackingWheelCfg(Matrix([['v_x3'], ['v_y3']]) * meter, symbols('o_3') * radian, symbols('r_3')* meter)

    cfg1 = TrackingWheelCfg(Matrix([-0.04445 * meter, 0.06985 * meter]), 0 * radian, 0.028297632 * meter)
    cfg2 = TrackingWheelCfg(Matrix([0.008382 * meter, -0.06985 * meter]), pi * radian, 0.028297632 * meter)
    cfg3 = TrackingWheelCfg(Matrix([-0.2032 * meter, -0.0127 * meter]), (3 * pi / 2) * radian, 0.028297632 * meter)

In [58]:
def body_delta_to_pod_delta(body_delta: Twist, pod_position: Matrix) -> Matrix:
    rot90ccw = Matrix([[0,-1],[1,0]])
    t = rot90ccw * pod_position.normalized()
    mag = pod_position.norm() * body_delta.dtheta
    rotational_contribution = (t*mag ) / radian # / radian bc we're going from angle to distance
    return Matrix([body_delta.dx, body_delta.dy]) + rotational_contribution

In [118]:
body_delta_to_pod_delta(Twist(symbols('dx'),symbols('dy') , symbols('d\\theta')), Matrix(['v_x','v_y'])* meter)

Matrix([
[-meter*d\theta*v_y/radian + dx],
[ meter*d\theta*v_x/radian + dy]])

In [127]:
xyc = body_delta_to_pod_delta(Twist(symbols('C_dx'),symbols('C_dy'), symbols('C_d\\theta')), Matrix(['v_x','v_y'])* meter)

In [128]:
def circ(radius):
    return 2 * pi * radius

In [129]:
def RotationToLinear(theta, radius):
    return theta * radius

In [130]:
xyc1 = xyc.subs('v_x', 'v_x1').subs('v_y', 'v_y1')
xyc2 = xyc.subs('v_x', 'v_x2').subs('v_y', 'v_y2')
xyc3 = xyc.subs('v_x', 'v_x3').subs('v_y', 'v_y3')

In [134]:
XYCx3 = Matrix([xyc1.T, xyc2.T, xyc3.T])

In [140]:
XYCx3

Matrix([
[-meter*C_d\theta*v_y1/radian + C_dx, meter*C_d\theta*v_x1/radian + C_dy],
[-meter*C_d\theta*v_y2/radian + C_dx, meter*C_d\theta*v_x2/radian + C_dy],
[-meter*C_d\theta*v_y3/radian + C_dx, meter*C_d\theta*v_x3/radian + C_dy]])

In [150]:
ChassisDelta = Matrix([symbols('C_dx'), symbols('C_dy'), symbols('C_d\\theta')])
ChassisDelta

Matrix([
[     C_dx],
[     C_dy],
[C_d\theta]])

In [152]:
Matrix([[1, 0, -symbols('v_y1')],[1, 0, -symbols('v_y2')],[1    , 0, -symbols('v_y3')]])

Matrix([
[1, 0, -v_y1],
[1, 0, -v_y2],
[1, 0, -v_y3]])

In [149]:
ChassisDelta

Matrix([
[     C_dx],
[     C_dy],
[C_d\theta]])

In [138]:
XYCx3 \
    .subs('C_dx', 2 * meter / second) \
    .subs('C_dy', 0) \
    .subs('C_d\\theta', 0)


Matrix([
[2*meter/second, 0],
[2*meter/second, 0],
[2*meter/second, 0]])

In [154]:
Matrix([['a','b','c'],['d','e','f'],['g','h','u']]) * Matrix(['x','y','z'])

Matrix([
[a*x + b*y + c*z],
[d*x + e*y + f*z],
[g*x + h*y + u*z]])

In [166]:
m = Matrix([
    [symbols('x_1') * radian / meter , symbols('y_1') * radian / meter , symbols('o_1') * radian / radian ],
    [symbols('x_2') * radian / meter , symbols('y_2') * radian / meter , symbols('o_2') * radian / radian ],
    [symbols('x_3') * radian / meter , symbols('y_3') * radian / meter , symbols('o_3') * radian / radian ]])
m

Matrix([
[radian*x_1/meter, radian*y_1/meter, o_1],
[radian*x_2/meter, radian*y_2/meter, o_2],
[radian*x_3/meter, radian*y_3/meter, o_3]])

In [178]:
Matrix([[1,0], [1,0]])

Matrix([
[1, 0],
[1, 0]])

In [176]:
M.inverse_ADJ()*Matrix(["e_1", "e_2", "e_3"])

Matrix([
[                                               -35*e_1*r_2/(-1225*r_1 - 1225*r_2) + 35*e_2*r_1/(-1225*r_1 - 1225*r_2)],
[35*e_1*r_3/(-1225*r_1 - 1225*r_2) + 35*e_2*r_3/(-1225*r_1 - 1225*r_2) + e_3*(-35*r_1 - 35*r_2)/(-1225*r_1 - 1225*r_2)],
[                                                   -1225*e_1/(-1225*r_1 - 1225*r_2) - 1225*e_2/(-1225*r_1 - 1225*r_2)]])

In [170]:
sympy.simplify(m.inverse_ADJ())

Matrix([
[meter*(-o_2*y_3 + o_3*y_2)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1)),  meter*(o_1*y_3 - o_3*y_1)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1)), meter*(-o_1*y_2 + o_2*y_1)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1))],
[ meter*(o_2*x_3 - o_3*x_2)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1)), meter*(-o_1*x_3 + o_3*x_1)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1)),  meter*(o_1*x_2 - o_2*x_1)/(radian*(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1))],
[                (x_2*y_3 - x_3*y_2)/(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1),                (-x_1*y_3 + x_3*y_1)/(o_1*x_2*y_3 - o_1*x_3*y_2 - o_2*x_1*y_3 + o_2*x_3*y_1 + o_3*x_1*y_2 - o_3*x_2*y_1),                 (x

## Updates

We can measure 1 axis at a pod, leaving one as an unknown

Swerve kinematics measure both